In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import cx_Oracle
from google.colab import files
from getpass import getpass
import functools
import operator

from pykospacing import Spacing
import kss

import warnings
warnings.filterwarnings('ignore')

## Colab Oracle Database 연결

In [ ]:
! ls -l oracle-instantclient*-basiclite-*.rpm || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm
! ls -l oracle-instantclient*-sqlplus-*.rpm   || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

-rw-r--r-- 1 root root 28553496 May  3  2019 oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm
-rw-r--r-- 1 root root 702792 May  3  2019 oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm


In [ ]:
! sudo apt-get install alien libaio1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
alien is already the newest version (8.95).
libaio1 is already the newest version (0.3.110-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
! sudo alien -i oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

	dpkg --no-force-overwrite -i oracle-instantclient19.3-basiclite_19.3.0.0.0-2_amd64.deb
(Reading database ... 157975 files and directories currently installed.)
Preparing to unpack oracle-instantclient19.3-basiclite_19.3.0.0.0-2_amd64.deb ...
Unpacking oracle-instantclient19.3-basiclite (19.3.0.0.0-2) over (19.3.0.0.0-2) ...
Setting up oracle-instantclient19.3-basiclite (19.3.0.0.0-2) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
! sudo alien -i --scripts oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

	dpkg --no-force-overwrite -i oracle-instantclient19.3-sqlplus_19.3.0.0.0-2_amd64.deb
(Reading database ... 157975 files and directories currently installed.)
Preparing to unpack oracle-instantclient19.3-sqlplus_19.3.0.0.0-2_amd64.deb ...
Unpacking oracle-instantclient19.3-sqlplus (19.3.0.0.0-2) over (19.3.0.0.0-2) ...
Setting up oracle-instantclient19.3-sqlplus (19.3.0.0.0-2) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
uploaded = files.upload()

Saving Wallet_thesis.zip to Wallet_thesis (1).zip


In [ ]:
!unzip -o -d /usr/lib/oracle/19.3/client64/lib/network/admin Wallet_thesis*.zip

Archive:  Wallet_thesis (1).zip
caution: filename not matched:  Wallet_thesis.zip


In [ ]:
!grep -A2 thesis /usr/lib/oracle/19.3/client64/lib/network/admin/tnsnames.ora

thesis_high = (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.ap-seoul-1.oraclecloud.com))(connect_data=(service_name=gf8a9fe6ffed6a6_thesis_high.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.ap-seoul-1.oraclecloud.com, OU=Oracle ADB SEOUL, O=Oracle Corporation, L=Redwood City, ST=California, C=US")))

thesis_low = (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.ap-seoul-1.oraclecloud.com))(connect_data=(service_name=gf8a9fe6ffed6a6_thesis_low.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.ap-seoul-1.oraclecloud.com, OU=Oracle ADB SEOUL, O=Oracle Corporation, L=Redwood City, ST=California, C=US")))

thesis_medium = (description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.ap-seoul-1.oraclecloud.com))(connect_data=(service_name=gf8a9fe6ffed6a6_thesis_medium.adb.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adb.ap-seoul-1.oraclecloud.com, OU=Or

In [ ]:
!cat /usr/lib/oracle/19.3/client64/network/admin/sqlnet.ora

cat: /usr/lib/oracle/19.3/client64/network/admin/sqlnet.ora: No such file or directory


In [ ]:
password = getpass('Enter the user password:')

Enter the user password:··········


In [ ]:
 ! sqlplus -L admin/$password@thesis_high <<<'show user'


SQL*Plus: Release 19.0.0.0.0 - Production on Wed Mar 30 16:44:24 2022
Version 19.3.0.0.0

Copyright (c) 1982, 2019, Oracle.  All rights reserved.

Last Successful login time: Wed Mar 30 2022 15:59:10 +00:00

Connected to:
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.15.0.1.0

SQL> USER is "ADMIN"
SQL> Disconnected from Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production
Version 19.15.0.1.0


In [ ]:
! sqlplus -s -L admin/$password@thesis_high @ script01.sql </dev/null

SP2-0310: unable to open file "script01.sql"


In [ ]:
con = cx_Oracle.connect("admin", password, "thesis_high")
cur = con.cursor()

In [ ]:
cur.execute("select TITLE from MID_CLASS where SEARCHQ='정보통신기술'")
title = cur.fetchall()

cur.execute("select SENTENCE from MID_CLASS where SEARCHQ='정보통신기술'")
sentence = cur.fetchall()

In [ ]:
thesis = pd.DataFrame({
    'title' : title,
    'sentence' : sentence})

thesis['sentence'] = functools.reduce(operator.add, (thesis['sentence']))

thesis = thesis.iloc[264:352, :]
thesis.head()

,title,sentence
264,"(이동통신시스템에 적용되는 AI 기술 동향.pdf,)",[이동통신시스템에 적용되는 AI 기술 동향요 약현 이동통신시스템에서 겪는 용량 스펙...
265,"(이중 기능 레이다-통신 최신 기술 연구 동향.pdf,)",[논문 20451018 The Journal of Korean Institute o...
266,(산업연관분석을 이용한 과기출연연 경제적 기여효과에 대한 연구 한국생산기술연구원 ...,[1 KOTIS Journal of Korea Technology Innovatio...
267,"(산업연관분석을 이용한 정보보호 산업의 경제 파급효과 분석.pdf,)",[논문 20130172한국정보전자통신기술학회논문지jkiiect202 Vol.13 N...
268,"(상담 및 심리치료에서 인공지능 기술의 활용 국외사례를 중심으로.pdf,)",[821 상담 및 심리치료에서 인공지능 기술의 활용국외사례를 중심으로 김 도 연 조...


## 띄어쓰기 전처리

In [ ]:
spacing = Spacing()

thesis['res']='null'

for word2 in thesis.columns : 
    thesis['sentence'] = thesis['sentence'].astype("string")

def pre1(x) : 
  x = x.strip('['']')
  x = x.replace(" ", '')
  return spacing(x)

thesis['sentence'] = thesis['sentence'].transform(pre1)
thesis

,title,sentence,res
264,"(이동통신시스템에 적용되는 AI 기술 동향.pdf,)",이동통신 시스템에 적용되는 AI 기술동향 요약현이동통신 시스템에서 겪는 용량스펙트럼...,null
265,"(이중 기능 레이다-통신 최신 기술 연구 동향.pdf,)",논문 20451018TheJ ournalofKoreanInstituteofCommu...,null
266,(산업연관분석을 이용한 과기출연연 경제적 기여효과에 대한 연구 한국생산기술연구원 ...,1KOTISJ ournalofKoreaTechnology InnovationSoci...,null
267,"(산업연관분석을 이용한 정보보호 산업의 경제 파급효과 분석.pdf,)",논문 20130172 한국정보전자통신기술학회 논문 지jkiiect202Vol.13N...,null
268,"(상담 및 심리치료에서 인공지능 기술의 활용 국외사례를 중심으로.pdf,)",821 상담 및 심리치료에서 인공지능 기술의 활용국외 사례를 중심으로 김도 연 조민...,null
...,...,...,...
347,"(통신산업 기술혁신에 영향을 미치는 요인에 관한 연구.pdf,)",Inform ationSociety MediaApril2019 Vol.20No.11...,null
348,"(드론 무선 통신 기술.pdf,)",요약본고에서는 드론이 초연결화 및 초지능화와 결합되어 4차 산업혁명의 핵심기술들이 ...,null
349,"(전기 통신 사업 분류 제도 개선 방안에 관한 연구.pdf,)",논문 19440218TheJ ournalofKoreanInstituteofCommu...,null
350,"(전달 노드의 특성을 이용한 효과적인 지연 허용 네트워크 라우팅 프로토콜.pdf,)",Jou rnalof KIIT.Vol.17No.11pp.5965Nov.302019.p...,null


## 논문 결론 추출

In [ ]:
def fd(x) : 
  if (x.find("결론") != -1) & (x.find("참고문헌") != -1) :
    return x[x.find("결론"):x.find("참고문헌")]
  elif (x.find("결론") != -1) & (x.find("Reference") != -1) :
    return x[x.find("결론"):x.find("Reference")]
  elif (x.find("결론") != -1) & (x.find("참고문헌") == -1) & (x.find("Reference") == -1): 
    return x[x.find("결론"):]
  elif (x.find("결과") != -1) & (x.find("참고문헌") != -1) : 
    return x[x.find("결과"):x.find("참고문헌")]
  elif (x.find("결과") != -1) & (x.find("Reference") != -1) :
    return x[x.find("결과"):x.find("Reference")]
  elif (x.find("결과") != -1) & (x.find("참고문헌") == -1) & (x.find("Reference") == -1): 
    return x[x.find("결과"):]
  elif (x.find("참고문헌") != -1) :
    return x[0:x.find("참고문헌")]
  elif (x.find("Reference") != -1) :
    return x[0:x.find("reference")]
  else :
    return x

thesis['res'] = thesis['sentence'].transform(fd)

def refd(x) : 
    if (len(x[1])/2)<len(x[2]) :
        if x[2].find("결론",2) != -1 :
            return x[2][x[2].find("결론",2):]
        elif x[2].find("결과",2) != -1 :
            return x[2][x[2].find("결과",2):]
        else :
            return x[2]
    else :
        return x[2]

thesis['result'] = thesis.apply(refd, axis=1)
thesis = thesis.drop(['res'], axis=1)
thesis

,title,sentence,result
264,"(이동통신시스템에 적용되는 AI 기술 동향.pdf,)",이동통신 시스템에 적용되는 AI 기술동향 요약현이동통신 시스템에서 겪는 용량스펙트럼...,"결론본고에서는 AI 기반이동통신 시스템 기술과 산학연 동향을 살펴보았다., 2030..."
265,"(이중 기능 레이다-통신 최신 기술 연구 동향.pdf,)",논문 20451018TheJ ournalofKoreanInstituteofCommu...,결론 및 향후 발전방향 무선주파수를 사용하는 레이다와 무선통신 시스템 사이에 주파수...
266,(산업연관분석을 이용한 과기출연연 경제적 기여효과에 대한 연구 한국생산기술연구원 ...,1KOTISJ ournalofKoreaTechnology InnovationSoci...,결론 본 연구에서는 생기원의 지난 24년간 연구활동으로 인한 수혜 기업의 경제적 성...
267,"(산업연관분석을 이용한 정보보호 산업의 경제 파급효과 분석.pdf,)",논문 20130172 한국정보전자통신기술학회 논문 지jkiiect202Vol.13N...,결론 본 논문에서는 33개의 산업으로 구성되어 있는 2015년 실측 표 기준산업연관...
268,"(상담 및 심리치료에서 인공지능 기술의 활용 국외사례를 중심으로.pdf,)",821 상담 및 심리치료에서 인공지능 기술의 활용국외 사례를 중심으로 김도 연 조민...,결과 우울증상이 감소한 것으로 나타난 연구결과 또한 발표된 바 있다 Fitzpat ...
...,...,...,...
347,"(통신산업 기술혁신에 영향을 미치는 요인에 관한 연구.pdf,)",Inform ationSociety MediaApril2019 Vol.20No.11...,결론 및 논의 본 연구는 1980년부터 2014년까지 42개국의 통신산업기술혁신 관...
348,"(드론 무선 통신 기술.pdf,)",요약본고에서는 드론이 초연결화 및 초지능화와 결합되어 4차 산업혁명의 핵심기술들이 ...,결론적으로 비관 제공 역에서 무인비행장치의 비가 시권운용을 위해서는 현재 드론에서 ...
349,"(전기 통신 사업 분류 제도 개선 방안에 관한 연구.pdf,)",논문 19440218TheJ ournalofKoreanInstituteofCommu...,결론 분석에서 제시된 각 안의 가치와 평가기 디멘2에 한정성 평가를 종합하여 평가기...
350,"(전달 노드의 특성을 이용한 효과적인 지연 허용 네트워크 라우팅 프로토콜.pdf,)",Jou rnalof KIIT.Vol.17No.11pp.5965Nov.302019.p...,결론 본 논문에서는 보행자보다 더 큰 버퍼 크기를 가지고 더 고속으로 넓은 범위를 ...


In [ ]:
thesis.to_excel(excel_writer='논문결론.xlsx')

## 문장 분리

In [ ]:
thesis = pd.read_excel('논문결론.xlsx')

def pre2(x) : 
    x = x.strip("('""',)")
    return x

thesis['title'] = thesis['title'].transform(pre2)
thesis = thesis.drop(['sentence'], axis=1)

resdf = pd.DataFrame({
    'title' : [],
    'sentence' : []
})

def spl(df) : 
  if len(df[1])<10000:
    global resdf
    res = kss.split_sentences(df[1])
    title = []
    for i in res:
      title.append(df[0])
    plusdf = pd.DataFrame({
        'title' : title,
        'sentence' : res
    })
    resdf = pd.concat([resdf, plusdf])
    
thesis.apply(spl, axis=1)
resdf

,title,sentence
0,이동통신시스템에 적용되는 AI 기술 동향.pdf,결론본고에서는 AI 기반이동통신 시스템 기술과 산학연 동향을 살펴보았다.
1,이동통신시스템에 적용되는 AI 기술 동향.pdf,2030년 도래할 6G의 지상과 가상세계에서의 초 신뢰 초저지연초 성능 서비스 공중...
2,이동통신시스템에 적용되는 AI 기술 동향.pdf,AI 기반 무선 네트워크 기술은 셀 구성 최적화와 자원관리 측면의 이점이 있어 현 ...
3,이동통신시스템에 적용되는 AI 기술 동향.pdf,그러나 AI 기반 무선 전송 기술은 학습 환경과 실제 환경간 불일치로 인한 성능저하...
0,이중 기능 레이다-통신 최신 기술 연구 동향.pdf,결론 및 향후 발전방향 무선주파수를 사용하는 레이다와 무선통신 시스템 사이에 주파수...
...,...,...
13,전기 통신 사업 분류 제도 개선 방안에 관한 연구.pdf,정성평가기을 반한 종합 결과를 고려하면 재의 3구분의 역무구분은 2가지로 통합하는 ...
14,전기 통신 사업 분류 제도 개선 방안에 관한 연구.pdf,본 연구를 통해 정책담당자와 정책상자의의 차이가 많이 좁지기는 했으나 그 차이가 여...
0,전달 노드의 특성을 이용한 효과적인 지연 허용 네트워크 라우팅 프로토콜.pdf,결론 본 논문에서는 보행자보다 더 큰 버퍼 크기를 가지고 더 고속으로 넓은 범위를 ...
1,전달 노드의 특성을 이용한 효과적인 지연 허용 네트워크 라우팅 프로토콜.pdf,성능 분석 결과 제안 프로토콜은 수신노드의 메시지 전달 기준을 차량과 보행자 노드와...


In [ ]:
thesis.to_excel('결론추출.xlsx')